In [1]:
import datetime
import pandas as pd
import pyaurorax

aurorax = pyaurorax.PyAuroraX()

# AuroraX ML-enhanced searching

Integrated with the AuroraX search engine, there is the ability to search through ephemeris and conjunctions based on various machine learning model metadata that has been uploaded to the platform. 

Two use cases to highlight, but there are many more: 
  - finding all conjunctions between spacecrafts and the THEMIS ASIs where an ML model believes it is not cloudy in the THEMIS ASI
  - retrieve all 1-minute times where an ML model believes THEMIS ASIs have Amorphous Pulsating Aurora (APA) in the field of view.

More information about the available ML metadata can be found [here](https://docs.aurorax.space/ml/overview).

# Conjunction Searching

Let's first have a look at how to do conjunction searches, and find results by leveraging some ML-derived metadata.

## UCalgary Amorphous Pulsating Aurora (APA) model

UCalgary has developed a machine learning model for identifying Amorphous Pulsating Aurora (APA) in the THEMIS all-sky imagers. This is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Metadata fields in AuroraX can be searched upon, allowing users to filter results based on them. Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where this particular model thinks there is APA in the field-of-view for that camera, over a 1 month period.

More information about this model can be found [here](https://docs.aurorax.space/ml/models/ucalgary_apa/).

In [2]:
# set timeframe, distance, and conjunction type
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 31, 23, 59, 59)
conjunction_types = [aurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# set ground criteria block
ground = [
    aurorax.search.GroundCriteriaBlock(
        programs=["themis-asi"],
        metadata_filters=aurorax.search.MetadataFilter(expressions=[
            # only find records that were classified as APA
            aurorax.search.MetadataFilterExpression("calgary_apa_ml_v1", "classified as APA", operator="="),

            # with a confidence of at least 95%
            aurorax.search.MetadataFilterExpression("calgary_apa_ml_v1_confidence", 95, operator=">="),
        ]))
]

# set space criteria block
space = [aurorax.search.SpaceCriteriaBlock(programs=["themis"], hemisphere=["northern"])]

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground,
    space=space,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2025-01-27 13:55:52.287999] Search object created
[2025-01-27 13:55:52.324823] Request submitted
[2025-01-27 13:55:52.324858] Request ID: 4d74443f-b37e-4f91-af5b-93c0b2938f9b
[2025-01-27 13:55:52.324869] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/4d74443f-b37e-4f91-af5b-93c0b2938f9b
[2025-01-27 13:55:52.324878] Waiting for data ...
[2025-01-27 13:55:53.758284] Checking for data ...
[2025-01-27 13:55:54.206215] Data is now available
[2025-01-27 13:55:54.206426] Retrieving data ...
[2025-01-27 13:55:54.685081] Retrieved 52.8 kB of data containing 8 records


In [3]:
# show the conjunction results
#
# NOTE: while here we format the results into a Pandas dataframe, this
# is not required. We actually don't include Pandas as a dependency since
# it's used simply as a nice add-on to view data. If you're good with slicing
# and dicing lists and dictionaries, you'll be fine without it.
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
0,nbtrace,2008-01-01T11:30:00,2008-01-01T11:36:00,"[DataSource(identifier=45, program='themis-asi...",383.962128,428.502480,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-01T11:30:00,2008-01-01T11:36:00
1,nbtrace,2008-01-01T11:30:00,2008-01-01T11:59:00,"[DataSource(identifier=45, program='themis-asi...",340.029413,499.355723,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-01T11:30:00,2008-01-01T11:59:00
5,nbtrace,2008-01-08T05:50:00,2008-01-08T05:52:00,"[DataSource(identifier=49, program='themis-asi...",485.213758,499.938201,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-08T05:50:00,2008-01-08T05:52:00
3,nbtrace,2008-01-14T04:50:00,2008-01-14T04:59:00,"[DataSource(identifier=49, program='themis-asi...",150.330448,200.986916,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-14T04:50:00,2008-01-14T04:59:00
6,nbtrace,2008-01-14T04:50:00,2008-01-14T04:59:00,"[DataSource(identifier=49, program='themis-asi...",203.965929,271.159251,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-14T04:50:00,2008-01-14T04:59:00
4,nbtrace,2008-01-19T08:02:00,2008-01-19T08:19:00,"[DataSource(identifier=51, program='themis-asi...",318.807754,390.216280,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-19T08:05:00,2008-01-19T08:19:00
2,nbtrace,2008-01-19T08:04:00,2008-01-19T08:09:00,"[DataSource(identifier=46, program='themis-asi...",393.493000,494.091856,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-19T08:09:00,2008-01-19T08:04:00
7,nbtrace,2008-01-25T06:40:00,2008-01-25T06:49:00,"[DataSource(identifier=54, program='themis-asi...",327.930714,372.530865,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-25T06:40:00,2008-01-25T06:49:00


With the search now done and saved to the 's' variable, we can use the Swarm-Aurora helper functions to further examine the results.

NOTE: some IDEs can sometimes have issues figuring out the browser to use when opening Swarm-Aurora. To help with this, you can manually specify the 'browser' parameter to the helper function with one of the available values specified in the official `webbrowser` [module documentation](https://docs.python.org/3/library/webbrowser.html#webbrowser.get).

In [4]:
# open browser to Swarm-Aurora for this search
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s)

# if you want to force it to use Google Chrome, you can do the following
#
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s, browser="google-chrome")

# alternatively, you can also just get the web URL for Swarm-Aurora for this
# specific request, and copy paste it into the browser manually.
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=4d74443f-b37e-4f91-af5b-93c0b2938f9b


## UCalgary cloud model

UCalgary has developed a machine learning model for identifying cloud in the THEMIS all-sky imagers. This is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where this particular model thinks there is cloud in the field-of-view at Gillam, over a 7 day period.

More information about this model can be found [here](https://docs.aurorax.space/ml/models/ucalgary_cloud/).

In [5]:
# set timeframe, distance, and conjunction type
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 15, 23, 59, 59)
conjunction_types = [aurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# set ground criteria block
ground = [
    aurorax.search.GroundCriteriaBlock(
        programs=["themis-asi"],
        metadata_filters=aurorax.search.MetadataFilter(expressions=[
            # only find records that were classified as not cloud
            aurorax.search.MetadataFilterExpression("calgary_cloud_ml_v1", "classified as not cloudy", operator="="),

            # with a confidence of at least 75%
            aurorax.search.MetadataFilterExpression("calgary_cloud_ml_v1_confidence", 75, operator=">=")
        ]))
]

# set space criteria block
space = [aurorax.search.SpaceCriteriaBlock(programs=["swarm", "elfin"], hemisphere=["northern"])]

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground,
    space=space,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2025-01-27 13:55:56.422216] Search object created
[2025-01-27 13:55:56.448051] Request submitted
[2025-01-27 13:55:56.448123] Request ID: bf9550fe-ba7b-4b74-9b04-33657bbd2497
[2025-01-27 13:55:56.448151] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/bf9550fe-ba7b-4b74-9b04-33657bbd2497
[2025-01-27 13:55:56.448174] Waiting for data ...
[2025-01-27 13:55:57.879891] Checking for data ...
[2025-01-27 13:55:59.305871] Checking for data ...
[2025-01-27 13:55:59.739217] Data is now available
[2025-01-27 13:55:59.739443] Retrieving data ...
[2025-01-27 13:56:00.410114] Retrieved 2.2 MB of data containing 270 records


In [6]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
160,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",206.588207,260.819127,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
48,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",192.029390,336.631805,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
89,nbtrace,2020-01-01T01:45:00,2020-01-01T01:46:00,"[DataSource(identifier=44, program='themis-asi...",205.885002,420.474697,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T01:46:00,2020-01-01T01:45:00
1,nbtrace,2020-01-01T01:46:00,2020-01-01T01:47:00,"[DataSource(identifier=44, program='themis-asi...",284.420850,493.197476,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T01:46:00,2020-01-01T01:47:00
113,nbtrace,2020-01-01T03:15:00,2020-01-01T03:16:00,"[DataSource(identifier=47, program='themis-asi...",385.355069,414.231784,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T03:16:00,2020-01-01T03:15:00
17,nbtrace,2020-01-01T03:15:00,2020-01-01T03:16:00,"[DataSource(identifier=47, program='themis-asi...",412.812856,489.362899,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T03:16:00,2020-01-01T03:15:00
253,nbtrace,2020-01-01T04:07:00,2020-01-01T04:08:00,"[DataSource(identifier=53, program='themis-asi...",343.134412,396.842659,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T04:07:00,2020-01-01T04:08:00
68,nbtrace,2020-01-01T04:49:00,2020-01-01T04:50:00,"[DataSource(identifier=65, program='themis-asi...",323.719101,352.655457,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T04:49:00,2020-01-01T04:50:00
190,nbtrace,2020-01-01T04:49:00,2020-01-01T04:50:00,"[DataSource(identifier=65, program='themis-asi...",244.886114,355.591219,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T04:49:00,2020-01-01T04:50:00
217,nbtrace,2020-01-01T05:18:00,2020-01-01T05:19:00,"[DataSource(identifier=53, program='themis-asi...",371.020465,478.895130,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T05:19:00,2020-01-01T05:18:00


In [7]:
# have a look at the results in Swarm-Aurora
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=bf9550fe-ba7b-4b74-9b04-33657bbd2497


## Oslo Aurora THEMIS (OATH) model

The OATH model was developed by [Clausen & Nickisch](https://doi.org/10.1029/2018JA025274), and has been run by the AuroraX team for all THEMIS ASI data in the AuroraX search engine.

- Paper: https://doi.org/10.1029/2018JA025274
- More information: http://tid.uio.no/plasma/oath

This paper has two versions of the model, one that is a binary classifier (aurora/no aurora), and one that is a 6-class classifier. AuroraX currently only contains metadata derived from the 6-class model.

The OATH mode is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where the OATH model has classified the minute as diffuse or discrete aurora, over a 1 day period.

More information about the AuroraX utilization of this model can be found [here](https://docs.aurorax.space/ml/models/clausen_oath/).

In [8]:
# set timeframe, distance, and conjunction types
start = datetime.datetime(2015, 1, 1, 0, 0, 0)
end = datetime.datetime(2015, 1, 31, 23, 59, 59)
conjunction_types = [pyaurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# set ground criteria block
ground = [
    aurorax.search.GroundCriteriaBlock(
        programs=["themis-asi"],
        metadata_filters=aurorax.search.MetadataFilter(expressions=[
            aurorax.search.MetadataFilterExpression(
                "clausen_ml_oath",
                [
                    # only find records that have one of these three classifications
                    "classified as diffuse",
                    "classified as discrete",
                    "classified as arc"
                ],
                operator="in")
        ]))
]

# set space criteria block
space = [aurorax.search.SpaceCriteriaBlock(programs=["dmsp"], hemisphere=["northern"])]

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground,
    space=space,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2025-01-27 13:56:01.049768] Search object created
[2025-01-27 13:56:01.074749] Request submitted
[2025-01-27 13:56:01.074788] Request ID: b1559ccc-b895-4eab-bd4d-4aac7314b4ff
[2025-01-27 13:56:01.074818] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/b1559ccc-b895-4eab-bd4d-4aac7314b4ff
[2025-01-27 13:56:01.074829] Waiting for data ...
[2025-01-27 13:56:02.499857] Checking for data ...
[2025-01-27 13:56:03.940264] Checking for data ...
[2025-01-27 13:56:04.371165] Data is now available
[2025-01-27 13:56:04.371354] Retrieving data ...
[2025-01-27 13:56:04.996504] Retrieved 1.5 MB of data containing 222 records


In [9]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
166,nbtrace,2015-01-01T06:38:00,2015-01-01T06:40:00,"[DataSource(identifier=71, program='themis-asi...",129.676030,477.204038,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T06:39:00,2015-01-01T06:38:00
98,nbtrace,2015-01-01T10:02:00,2015-01-01T10:02:00,"[DataSource(identifier=51, program='themis-asi...",38.161652,38.161652,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T10:02:00,2015-01-01T10:02:00
86,nbtrace,2015-01-01T10:58:00,2015-01-01T10:58:00,"[DataSource(identifier=48, program='themis-asi...",424.850074,424.850074,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T10:58:00,2015-01-01T10:58:00
143,nbtrace,2015-01-01T13:36:00,2015-01-01T13:36:00,"[DataSource(identifier=53, program='themis-asi...",477.845592,477.845592,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T13:36:00,2015-01-01T13:36:00
63,nbtrace,2015-01-01T14:18:00,2015-01-01T14:19:00,"[DataSource(identifier=45, program='themis-asi...",429.648698,469.288619,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T14:19:00,2015-01-01T14:18:00
148,nbtrace,2015-01-01T16:00:00,2015-01-01T16:00:00,"[DataSource(identifier=57, program='themis-asi...",352.260006,352.260006,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T16:00:00,2015-01-01T16:00:00
207,nbtrace,2015-01-01T20:10:00,2015-01-01T20:11:00,"[DataSource(identifier=71, program='themis-asi...",285.473795,299.878087,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T20:10:00,2015-01-01T20:11:00
28,nbtrace,2015-01-01T21:31:00,2015-01-01T21:33:00,"[DataSource(identifier=71, program='themis-asi...",75.318471,425.663394,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T21:32:00,2015-01-01T21:33:00
165,nbtrace,2015-01-02T06:23:00,2015-01-02T06:24:00,"[DataSource(identifier=71, program='themis-asi...",335.819298,365.536067,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-02T06:23:00,2015-01-02T06:24:00
192,nbtrace,2015-01-02T09:00:00,2015-01-02T09:00:00,"[DataSource(identifier=71, program='themis-asi...",356.122232,356.122232,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-02T09:00:00,2015-01-02T09:00:00


In [10]:
# have a look at the results in Swarm-Aurora
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=b1559ccc-b895-4eab-bd4d-4aac7314b4ff


# Ephemeris searching

Let's next have a look at doing ephemeris searches and leverage the ML-derived metadata.

## UCalgary Amorphous Pulsating Aurora (APA) model

In [11]:
# set timeframe and program values
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 31, 23, 59, 59)
programs = ["themis-asi"]

# to filter based the ML model data, we need to perform the ephemeris
# search using metadata filters.
#
# below, we're going to filter to only retrieve ephemeris data for when
# the 'calgary_apa_ml_v1' field says 'classified as APA', and has a
# confidence score greater than or equal to 95%.
metadata_filters_logical_operator = "AND"
metadata_filters = [
    {
        "key": "calgary_apa_ml_v1",
        "operator": "in",
        "values": [
            "classified as APA"  # only find records that were classified as APA
        ]
    },
    {
        "key": "calgary_apa_ml_v1_confidence",
        "operator": ">=",
        "values": [
            "95"  # with a confidence of at least 95%
        ]
    }
]

# perform the search
s = aurorax.search.ephemeris.search(start=start,
                                    end=end,
                                    programs=programs,
                                    metadata_filters_logical_operator=metadata_filters_logical_operator,
                                    metadata_filters=metadata_filters,
                                    verbose=True)

[2025-01-27 13:56:06.485625] Search object created
[2025-01-27 13:56:06.518284] Request submitted
[2025-01-27 13:56:06.518356] Request ID: 402d5e90-7870-4bd8-a6d6-f7527444663b
[2025-01-27 13:56:06.518382] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/402d5e90-7870-4bd8-a6d6-f7527444663b
[2025-01-27 13:56:06.518405] Waiting for data ...
[2025-01-27 13:56:07.933586] Checking for data ...
[2025-01-27 13:56:08.350751] Data is now available
[2025-01-27 13:56:08.350983] Retrieving data ...
[2025-01-27 13:56:10.411396] Retrieved 18.7 MB of data containing 5407 records


In [12]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("epoch")[0:10]

,data_source,epoch,location_geo,nbtrace,sbtrace,location_gsm,metadata
5117,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:50:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080707101569, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as no aurora',..."
5118,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:51:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080717793904, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5119,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:52:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080728486236, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as discrete', ..."
5120,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:53:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080739178574, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as discrete', ..."
5121,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:54:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.080807498709085, lon=162.1667...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5122,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:55:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.080807605632415, lon=162.1667...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as moon', 'cal..."
5123,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:56:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080771255578, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5124,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:57:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.080807819479084, lon=162.1667...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5125,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:58:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080792640243, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5126,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:59:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080803332583, lon=162.16677...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."


In [13]:
# print the results of the first 10 ephemeris records as a table showing just
# the classification and confidence score
#
# NOTE: you can use Pandas here like earlier, but here's an example of looking
# at the results without it
print(f"{'Timestamp':<{25}}"
      f"{'calgary_apa_ml_v1':<{25}} "
      f"{'confidence':<{18}}")
print("====================================================================")
for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{25}}"
          f"{s.data[i].metadata['calgary_apa_ml_v1']:<{23}} "  # type: ignore
          f"  {s.data[i].metadata['calgary_apa_ml_v1_confidence']:<{18}} ")  # type: ignore

Timestamp                calgary_apa_ml_v1         confidence        
2008-01-01 11:30:00      classified as APA         99.96              
2008-01-01 11:31:00      classified as APA         99.96              
2008-01-01 11:32:00      classified as APA         99.96              
2008-01-01 11:33:00      classified as APA         99.96              
2008-01-01 11:34:00      classified as APA         99.96              
2008-01-01 11:35:00      classified as APA         99.96              
2008-01-01 11:36:00      classified as APA         99.96              
2008-01-01 11:37:00      classified as APA         99.96              
2008-01-01 11:38:00      classified as APA         99.96              
2008-01-01 11:39:00      classified as APA         99.96              


## UCalgary cloud model

In [14]:
# set up search parameters
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 7, 23, 59, 59)
programs = ["themis-asi"]
platforms = ["gillam"]

# we're going to filter to only retrieve ephemeris data for when
# the 'calgary_cloud_ml_v1' field says 'classified as not cloudy',
# and has a confidence score greater than or equal to 75%.
metadata_filters_logical_operator = "AND"
metadata_filters = [{
    "key": "calgary_cloud_ml_v1",
    "operator": "in",
    "values": ["classified as not cloudy"]
}, {
    "key": "calgary_cloud_ml_v1_confidence",
    "operator": ">",
    "values": ["75"]
}]

# Perform the search
s = aurorax.search.ephemeris.search(start=start,
                                    end=end,
                                    programs=programs,
                                    platforms=platforms,
                                    metadata_filters_logical_operator=metadata_filters_logical_operator,
                                    metadata_filters=metadata_filters,
                                    verbose=True)

[2025-01-27 13:56:25.715577] Search object created
[2025-01-27 13:56:25.740650] Request submitted
[2025-01-27 13:56:25.740722] Request ID: cdcab412-ed61-4e18-8297-2e889e5f8c64
[2025-01-27 13:56:25.740748] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/cdcab412-ed61-4e18-8297-2e889e5f8c64
[2025-01-27 13:56:25.740772] Waiting for data ...
[2025-01-27 13:56:27.151927] Checking for data ...
[2025-01-27 13:56:27.563638] Data is now available
[2025-01-27 13:56:27.563766] Retrieving data ...
[2025-01-27 13:56:27.868539] Retrieved 2.1 MB of data containing 597 records


In [15]:
# Again, print some results
print(f"{'Timestamp':<{25}}"
      f"{'Classification':<{30}} "
      f"{'Confidence':<{18}}")
print("====================================================================")
for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{25}}"
          f"{s.data[i].metadata['calgary_cloud_ml_v1']:<{28}} "  # type: ignore
          f"  {s.data[i].metadata['calgary_cloud_ml_v1_confidence']:<{18}} ")  # type: ignore

Timestamp                Classification                 Confidence        
2008-01-01 23:28:00      classified as not cloudy       91.81              
2008-01-01 23:29:00      classified as not cloudy       91.81              
2008-01-02 00:00:00      classified as not cloudy       99.96              
2008-01-02 00:01:00      classified as not cloudy       99.96              
2008-01-02 00:02:00      classified as not cloudy       99.96              
2008-01-02 00:03:00      classified as not cloudy       99.96              
2008-01-02 00:04:00      classified as not cloudy       99.96              
2008-01-02 00:05:00      classified as not cloudy       99.96              
2008-01-02 00:06:00      classified as not cloudy       99.96              
2008-01-02 00:07:00      classified as not cloudy       99.96              


## Oslo Aurora THEMIS (OATH) model

In [16]:
# set up search parameters
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 1, 23, 59, 59)
programs = ["themis-asi"]

# we're going to filter to only retrieve ephemeris data for when
# the 'clausen_ml_oath' field says 'classified as diffuse' or
# 'classified as discrete'
metadata_filters_logical_operator = "AND"
metadata_filters = [{
    "key": "clausen_ml_oath",
    "operator": "in",
    "values": [
        "classified as diffuse",
        "classified as discrete",
    ]
}]

# Perform the search
s = aurorax.search.ephemeris.search(start=start,
                                    end=end,
                                    programs=programs,
                                    metadata_filters_logical_operator=metadata_filters_logical_operator,
                                    metadata_filters=metadata_filters,
                                    verbose=True)

[2025-01-27 13:56:36.242458] Search object created
[2025-01-27 13:56:36.264890] Request submitted
[2025-01-27 13:56:36.264959] Request ID: 4fc8adc7-1ab5-4b48-abc3-cc0fc5160a64
[2025-01-27 13:56:36.264984] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/4fc8adc7-1ab5-4b48-abc3-cc0fc5160a64
[2025-01-27 13:56:36.265005] Waiting for data ...
[2025-01-27 13:56:37.678026] Checking for data ...
[2025-01-27 13:56:38.095498] Data is now available
[2025-01-27 13:56:38.095719] Retrieving data ...
[2025-01-27 13:56:38.706948] Retrieved 5.3 MB of data containing 1532 records


In [17]:
# Again, print some results
print(f"{'Timestamp':<{25}}"
      f"{'Classification':<{18}}")
print("=================================================")

for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{23}}"
          f"  {s.data[i].metadata['clausen_ml_oath']:<{18}} ")  # type: ignore

Timestamp                Classification    
2008-01-01 01:11:00      classified as discrete 
2008-01-01 01:19:00      classified as diffuse 
2008-01-01 01:20:00      classified as diffuse 
2008-01-01 01:22:00      classified as discrete 
2008-01-01 01:25:00      classified as diffuse 
2008-01-01 01:26:00      classified as diffuse 
2008-01-01 01:27:00      classified as diffuse 
2008-01-01 01:28:00      classified as diffuse 
2008-01-01 01:29:00      classified as diffuse 
2008-01-01 01:30:00      classified as diffuse 
